In [18]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr, wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests

In [3]:
clf_data = pd.read_csv('AUCs.txt', sep='\t', header=0)
clf_data.columns = ['datasets_name', 'C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']

In [5]:
clf_data.head()

,datasets_name,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [6]:
clf_data = clf_data.drop(['datasets_name'], axis=1)

In [19]:
clf_data.head()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,0.763,0.768,0.771,0.798
1,0.599,0.591,0.590,0.569
2,0.954,0.971,0.968,0.967
3,0.628,0.661,0.654,0.657
4,0.882,0.888,0.886,0.898


In [20]:
%%time 
comp_data = []

for i, lhs_column in enumerate(clf_data.columns):
    for j, rhs_column in enumerate(clf_data.columns):
        if i >= j:
            continue
        
        statistic, p = wilcoxon(clf_data[lhs_column], clf_data[rhs_column])
        comp_data.append([lhs_column, rhs_column, statistic, p])

Wall time: 4.98 ms


In [21]:
clf_data_comp = pd.DataFrame.from_records(comp_data)
clf_data_comp.columns = ['param1', 'param2', 'stat', 'p']

In [22]:
clf_data_comp.head()

,param1,param2,stat,p
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826


In [23]:
clf_data_comp.sort_index(ascending=False, by='p')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,param1,param2,stat,p
1,C4.5,C4.5+cf,43.0,0.861262
4,C4.5+m,C4.5+m+cf,22.0,0.327826
3,C4.5+m,C4.5+cf,17.0,0.046333
5,C4.5+cf,C4.5+m+cf,10.0,0.022909
2,C4.5,C4.5+m+cf,11.0,0.015906
0,C4.5,C4.5+m,6.5,0.010757


In [24]:
reject, p_corrected, a1, a2 = multipletests(clf_data_comp.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [25]:
clf_data_comp['p_corrected'] = p_corrected
clf_data_comp['reject'] = reject

In [26]:
clf_data_comp.sort_index(ascending=False, by='p')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,param1,param2,stat,p,p_corrected,reject
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False


In [27]:
reject, p_corrected, a1, a2 = multipletests(clf_data_comp.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [28]:
clf_data_comp['p_corrected'] = p_corrected
clf_data_comp['reject'] = reject

In [29]:
clf_data_comp.sort_index(ascending=False, by='p')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,param1,param2,stat,p,p_corrected,reject
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
